## Inspect whether it is suitable/reasonable to use Pubtator species annotations from cited manuscripts for a dataset

Only a small subset of dataset records in NDE have values for the 'species' or 'infectiousAgent' fields. Many datasets have values for the 'citation' field. Pubtator allows [FTP downloads](https://ftp.ncbi.nlm.nih.gov/pub/lu/PubTatorCentral/) of taxonomic extractions/annotations by PMID. Can this approach be used to extrapolate this information for a dataset citing a PMID?

About the pubtator file: Each file contains five columns as shown in below:
0.   PMID:       PubMed abstract identifier
1.  Type:       i.e., gene, disease, chemical, species, and mutation
2.  Concept ID: Corresponding database identifier (e.g., NCBIGene ID, MESH ID)
3.   Mentions:   Bio-concept mentions corresponding to the PubMed abstract
4.  Resource:   Various manually annotated resources are included in the files (e.g., MeSH and gene2pubmed)


**Tested*
1. pull all records with a citation pmid
2. explode the dataframe to have only single value pmid instead of list pmids
3. load the Pubtator extracted species dataframe
4. get the intersect of the dataframes
5. Quick manual inspection for accuracy
  * Observations from quick manual inspection:
    - Pubtator annotates the entire manuscript which may include derived plant products as reagents in figures. This is a problematic source of error as the taxonomic ID of a reagent is irrelevant
6. Keep only taxon that have terms found in the dataset description (see potential improvements)

**Potential improvements**
- Throw out any identified species that are not mentioned in the dataset name or description. Since Pubtator gives the actual text that was mapped to an NCBI Taxon ID, a check can be performed to see if a taxa appears in the description or not and to throw it out if it doesn't.
- Note that since NCBI GEO and OMICS-DI have duplicate records, matching the name field, and comparing the lengths of the description fields will allow us to investigate whether the description field is better from GEO
  - If GEO does have better descriptions than its OMICS-DI duplicate, then a species may successfully map to the GEO version but fail in matching the OMICS-DI version
  
**Potential other applications**
Pubtator also has disease extraction, however, they align to MeSH which is not a true disease ontology. Pubtator disease annotations can similarly be downloaded and the extracted disease terms for a pmid can be checked against a dataset description for the dataset citing that pmid. Once the healthconditions are obtained, they can be normalized using the Translator KPs.

In [2]:
import pandas as pd
from pandas import read_csv
import requests
import json
import time
import math

In [103]:
def extract_pmid(pmid_dict):
    try:
        pmid = str(pmid_dict['pmid'])
    except:
        pmid = pmid_dict.replace('{pmid: ','').replace('}','')
    return pmid

def confirm_result(row):
    truthcheck = []
    try:
        taxalist = row['taxname'].split('|')
        for eachtaxon in taxalist:
            if eachtaxon in row['description']:
                truthcheck.append('found')
            else:
                truthcheck.append('false')
        if 'found' in truthcheck:
            return 'yes'
        else:
            return 'no'
    except:
        return 'no'

### Fetch minimal metadata from only records with a citation pmid

In [74]:
%%time

## Perform the initial query

query_url = 'https://api.data.niaid.nih.gov/v1/query?q=_exists_:citation.pmid&fields=_id,name,description,citation.pmid&fetch_all=true'
r = requests.get(query_url)
cleanr = json.loads(r.text)
hits = cleanr['hits']
#print(len(cleanr['hits']))
df1 = pd.DataFrame(cleanr['hits'])
scroll_id = cleanr['_scroll_id']
total_hits = cleanr['total']

CPU times: total: 125 ms
Wall time: 1.07 s


In [15]:
%%time
## Scroll to get all the results

i = 0
#k = 3 
k = math.ceil(total_hits/1000)
while i < k:
    r2 = requests.get(f'https://api.data.niaid.nih.gov/v1/query?scroll_id={scroll_id}')
    tmp = json.loads(r2.text)
    scroll_id = tmp['_scroll_id']
    tmpdf = pd.DataFrame(tmp['hits'])
    df1 = pd.concat((df1,tmpdf),ignore_index=True)
    #print(len(df1))
    i = i+1
    time.sleep(0.25)

2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
159000


KeyError: '_scroll_id'

In [16]:
## inspect the results
print(df1.head(n=3))

                   _id  _score                citation  \
0  OMICSDI_PRJNA775608     1.0  [{'pmid': '34874923'}]   
1   OMICSDI_PRJNA74531     1.0  [{'pmid': '23105075'}]   
2  OMICSDI_PRJNA754436     1.0  [{'pmid': '34793837'}]   

                                         description  \
0  Alveolar epithelial glycocalyx degradation med...   
1  Streptococcus agalactiae STIR-CD-17 Genome seq...   
2  CRISPR-Cas9 generated SARM1 knockout and epito...   

                                                name _ignored  
0  Alveolar epithelial glycocalyx degradation med...      NaN  
1                Streptococcus agalactiae STIR-CD-17      NaN  
2  CRISPR-Cas9 generated SARM1 knockout and epito...      NaN  


In [42]:
#### save the raw results
#df1.to_csv('data/citation_df_raw.tsv',sep='\t',header=0)

#### Clean up the results (since a single record may have multiple citations)
df2 = df1.explode('citation')
df3 = df2[['_id','citation','description','name']].copy()
df3.dropna(inplace=True)
print(len(df1),len(df2),len(df3))
df3['pmid'] = df3['citation'].apply(lambda x: extract_pmid(x))
df3.drop('citation',axis=1,inplace=True)
print(df3.head(n=2))

274648 284631 282321
                   _id                                        description  \
0  OMICSDI_PRJNA775608  Alveolar epithelial glycocalyx degradation med...   
1   OMICSDI_PRJNA74531  Streptococcus agalactiae STIR-CD-17 Genome seq...   

                                                name      pmid  
0  Alveolar epithelial glycocalyx degradation med...  34874923  
1                Streptococcus agalactiae STIR-CD-17  23105075  


In [43]:
#### save the clean results
#df3.to_csv('data/citation_df_clean.tsv',sep='\t',header=True)

### Use the citation pmids to pull species for those pmids from a pubtator export

In [ ]:
df3 = read_csv('data/citation_df_clean.tsv',delimiter='\t',header=0,index_col=0)

In [61]:
pmidlist = df3['pmid'].unique().tolist()
pmidintlist = []
faillist = []
for pmid in pmidlist:
    try:
        pmidintlist.append(int(pmid))
    except:
        faillist.append(pmid)

print(len(pmidlist), len(pmidintlist),len(faillist))

123965 123964 1


In [63]:
%%time
#### chunk the read of the input (turn it into a generator and iterate through it)
#### Only keep the species info from pubtator for pmids that have an NDE citation pmid match
speciesdf = pd.read_csv('data/species2pubtatorcentral.txt',delimiter='\t',
                        usecols=[0,2,3], names=['pmid','taxid','taxname'], header=None, chunksize=20000)
savedata = pd.DataFrame(columns=['pmid','taxid','taxname'])
for adf in speciesdf:
    tmpdf = adf.loc[adf['pmid'].isin(pmidlist)]
    tmpdf2 = adf.loc[adf['pmid'].isin(pmidintlist)]
    if len(tmpdf)>0:
        savedata = pd.concat((savedata,tmpdf),ignore_index=True)
    elif len(tmpdf2)>0:
        savedata = pd.concat((savedata,tmpdf2),ignore_index=True)

print(len(savedata))

675978
CPU times: total: 7min 18s
Wall time: 7min 21s


In [65]:
#### export the results so we don't have to do that again
savedata.to_csv('data/pmids_cited_taxa.tsv',sep='\t',header=0)

### Merge the NDE data (with citation pmids) with the Pubtator results 

In [91]:
#### Get rid of any duplication that was introduced as an artifact of the merging process
savedata['pmid'] = savedata['pmid'].astype(str)
merged_df = df3.merge(savedata,on='pmid',how='inner')
merged_df.drop_duplicates(keep='first',inplace=True)
print(len(merged_df))

1645100


In [92]:
#### Inspect the results
test_df = merged_df.head(n=20).copy()
print(test_df.head(n=2))

                   _id                                        description  \
0  OMICSDI_PRJNA775608  Alveolar epithelial glycocalyx degradation med...   
1  OMICSDI_PRJNA775608  Alveolar epithelial glycocalyx degradation med...   

                                                name      pmid  taxid  \
0  Alveolar epithelial glycocalyx degradation med...  34874923   4081   
1  Alveolar epithelial glycocalyx degradation med...  34874923  10090   

                               taxname  
0              Lycopersicon esculentum  
1  mice|Mice|Murine|Mouse|murine|mouse  


Pubtator will identify species in the full body of the paper for the pmid. This can include species that were used as reagents in the methodology (for example, Lycopersicon esculentum refers to some sort of tomato extract or protein that was mentioned ina figure). To ensure we only include species that were mentioned in the dataset, keep only taxa where at least one of the taxaname was mentioned in the record name or description

### Filter out species not mentioned in the record

In [98]:
#### Test the function for doing so

test_df['text match?'] = test_df.apply(lambda row: confirm_result(row), axis=1)
print(test_df.head(n=10))

                   _id                                        description  \
0  OMICSDI_PRJNA775608  Alveolar epithelial glycocalyx degradation med...   
1  OMICSDI_PRJNA775608  Alveolar epithelial glycocalyx degradation med...   
2  OMICSDI_PRJNA775608  Alveolar epithelial glycocalyx degradation med...   
3  OMICSDI_PRJNA775608  Alveolar epithelial glycocalyx degradation med...   
4  OMICSDI_PRJNA775608  Alveolar epithelial glycocalyx degradation med...   
5  OMICSDI_PRJNA775608  Alveolar epithelial glycocalyx degradation med...   
6        GEO_GSE186705  Acute Respiratory Distress Syndrome (ARDS) is ...   
7        GEO_GSE186705  Acute Respiratory Distress Syndrome (ARDS) is ...   
8        GEO_GSE186705  Acute Respiratory Distress Syndrome (ARDS) is ...   
9        GEO_GSE186705  Acute Respiratory Distress Syndrome (ARDS) is ...   

                                                name      pmid    taxid  \
0  Alveolar epithelial glycocalyx degradation med...  34874923     4081   
1 

In [104]:
#### Apply the function to determine if a record had a term that matched with at least one of the taxa names

merged_df['text match?'] = merged_df.apply(lambda row: confirm_result(row), axis=1)

In [105]:
#### Filter out the ones that didn't

good_df = merged_df.loc[merged_df['text match?']=='yes']
print(len(good_df))

233789


In [107]:
#### Inspect the resulting table
print(good_df.head(n=5))

                    _id                                        description  \
7         GEO_GSE186705  Acute Respiratory Distress Syndrome (ARDS) is ...   
8         GEO_GSE186705  Acute Respiratory Distress Syndrome (ARDS) is ...   
12   OMICSDI_PRJNA74531  Streptococcus agalactiae STIR-CD-17 Genome seq...   
13  OMICSDI_PRJNA754436  CRISPR-Cas9 generated SARM1 knockout and epito...   
14        GEO_GSE182091  The aim of this study was initially to determi...   

                                                 name      pmid  taxid  \
7   Alveolar epithelial glycocalyx degradation med...  34874923  10090   
8   Alveolar epithelial glycocalyx degradation med...  34874923   9606   
12                Streptococcus agalactiae STIR-CD-17  23105075   1311   
13  CRISPR-Cas9 generated SARM1 knockout and epito...  34793837  10090   
14  CRISPR-Cas9 generated SARM1 knockout and epito...  34793837  10090   

                                              taxname text match?  
7                 

In [109]:
#### Export the results so we don't have to do that again
good_df.to_csv('data/taxa_found.tsv',sep='\t',header=0)